### Hello fellow Scientists

This is a tutorial to understand how the Experiment and the Model classes work by means of an offline estimation. That means an estimation based on historical data when the bioprocess has already finished, so when no process is currently running > “offline”. This should be the first tutorial, understand it before you start with the online estimation examples. The biomoni package is built from numpy, pandas, scipy and sympy.

This tutorial shows how the biomoni package works and its possibilities. To actually understand the code, it is recommended to debug test.py within the Examples/offline_estimation directory.


In [1]:
#Imports
from biomoni.Experiment import Experiment
from biomoni.Yeast import Yeast
from biomoni.Yeast_variable_feedrate import Yeast_vf
from biomoni.visualize import visualize

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2


### Creating an Experiment object, a clean way to pre-process your data

By correctly instantiating an `Experiment` object, the raw data from one or several files are prepared and saved as object with several methods and attributes. The `Experiment` object can then later be used simulate or estimate with the `Model` classes of `biomoni` (e.g. `Yeast` or `Yeast_variable_feedrate`) or to visualize the data using the `visualize` module.

First start with a test file present in the “Messdaten” test folders (test, test2). To create an Experiment object its necessary to give at least these four parameters `path`, `exp_id`, `meta_path` and `types`. 
Where `path` describes a common path where ideally experiment data and metadata are located. `exp_id` is the id of the respective experiment. This is used for two things:
1. Find the right experiment data directory within path. E.g. the `test` directory which contains the `test.csv` data.
2. Use the respective settings in metadata where each row should have a unique exp_id.

`meta_path` is the location of the metadata within `path`. 
The `types` argument should be a dictionary with the different measurement types of the data (data from different measuring devices) as keys, where you can choose names arbitrarily, and the filenames as values. For example, if you have online data and CO2 data you could do something like this `{"on" : "online.csv", "CO2" : "CO2.csv"}`. First we look at the test data, we can do something like this `{"testi_test" : "test.csv"}`, where `test.csv` is the name of the file.

An assumption is that there is always a timestamp column in the measurement data which gives information about the time at which the data was recorded. Together with the time points `start` (desired process start) and `end` (end1, end2: desired process end) given in `metadata`, the measurement data is filtered according to these timestamps and based on this, the time is then created as a decimal number. First, the timestamp column (named TimeStamp in file) which contain string type values is always converted to a column called `ts` containing pandas.Timestamp type values. Then, the decimal time get's created (based on `ts`, `start` and `end`) and is automatically named `t` and is set as index in the dataframe. 

In [2]:
path = "../../Messdaten" #referenced from this location
exp_id = "test" #because the subfolder within "path" is named "test" and because the respective settings in metadata are named "test"
meta_path = "metadata.xlsx" #location of metadata within "path".
types = {"testi_test" : "test.csv"}

Exp = Experiment(path = path, exp_id = exp_id, meta_path = meta_path, types = types)
#show the data by printing Exp.dataset[type]
print("experimental data")
display(Exp.dataset["testi_test"])
#show metadata of the repsetive Experiment
print("metadata:")
display(Exp.metadata)

experimental data


,TimeStamp,col1,col2,col3,ts
t,,,,,
0.000000,24.11.2020 10:06,1,6,11,2020-11-24 10:06:00
0.900000,24.11.2020 11:00,2,7,12,2020-11-24 11:00:00
3.616667,24.11.2020 13:43,3,8,13,2020-11-24 13:43:00
4.250000,24.11.2020 14:21,4,9,14,2020-11-24 14:21:00


metadata:


start            2020-11-24 10:06:00
end1             2020-11-24 15:00:00
end2             2020-11-25 12:00:00
tsfeedOn         2020-11-24 10:29:00
feed_on                     0.383333
V0                               0.5
mX0_wet                          2.5
mX0                         0.672185
cX0                          1.34437
cS0                               10
drymassfactor               0.268874
mS0                              5.0
feed_rate                     0.0069
csf                              200
gas_flow                          30
T                                 32
M_base                             5
mE0                           0.0001
feed_factor                  0.00276
Name: test, dtype: object

Sometimes there is not the option to have one overall directory like `Messdaten`. Sometimes you have your experimental data and the metadata separated among different locations on your PC. For example, if you have something like the following scenario: a measurement device is creating a random folder on your device called `test2`, but the respective settings in metadata are still named `test`, you can use the argument `exp_dir_manual` to locate the experimental data. If experimental data and metadata are seperated the argument `path` refers to the directory where metadata is located.

In [3]:
exp_dir_manual = "../../Messdaten/test2"
Exp = Experiment(path = path, exp_id = exp_id, meta_path = meta_path, types = types, exp_dir_manual= exp_dir_manual)
print("experimental data")
display(Exp.dataset["testi_test"])

experimental data


,TimeStamp,col1,col2,col3,ts
t,,,,,
0.000000,24.11.2020 10:06,16,21,26,2020-11-24 10:06:00
0.900000,24.11.2020 11:00,17,22,27,2020-11-24 11:00:00
3.616667,24.11.2020 13:43,18,23,28,2020-11-24 13:43:00
4.250000,24.11.2020 14:21,19,24,29,2020-11-24 14:21:00


In [4]:
#dropping columns from the data
Exp.drop_col({"testi_test" : ["col1", "col3"]})
display(Exp.dataset["testi_test"])

,TimeStamp,col2,ts
t,,,
0.000000,24.11.2020 10:06,21,2020-11-24 10:06:00
0.900000,24.11.2020 11:00,22,2020-11-24 11:00:00
3.616667,24.11.2020 13:43,23,2020-11-24 13:43:00
4.250000,24.11.2020 14:21,24,2020-11-24 14:21:00


In [5]:
#detelting whole data types from the data
Exp.pop_dataframe(["testi_test", "lel"])
display(Exp.dataset)

/home/paul/pCloudDrive/Code/biomoni/biomoni/Experiment.py:350: UserWarning: The type 'lel' must be in the dataset
  warnings.warn("The type '{0}' must be in the dataset".format(typ))


{}

### Real measurement data from different sources

The example above was just a minimal example, let's work with real measurement data. In the next example we have measurement data coming from three different sources: the Sartorius Process Unit generating online measurement values of the process(`on`), the BlueSens CO2 Sensor collecting CO2 vol% values (`CO2`) and a hand made Excel file containing the offline measured data (`off`). In order to read the data, we need to in instantiate an Experiment object using more information than `path, exp_id, meta_path, types`. Since the data from different devices is differently formatted, we need to give information about this formatting. The constructor needs the information where the TimeStamp column is. In our case, the TimeStamp column is always the first column (`0`). This would be also the default value if no `index_ts` argument is given, so this is kinda redundant here. The `read_csv_settings` are the respective settings to read the data into a pandas DataFrame. The argument `to_datetime_settings` is used to recognize the correct DateTime format with pandas. The argument `calc_rate` is used to calculate the 1. derivation from a column. The argument `endpoint` tells which endpoint column to be used in `metadata`, if there are several endpoints. The argument `read_excel_settings` tells the settings to read the metadata from an Excel file (constraint: `metadata` has to be present in an Excel file). The smooth argument can be used to create smoothed values from the existing columns, using the pandas `ewm` method and `kwargs_smooth` contains the respective smoothing settings.

The biomoni package comes with a module called `visualize` which can be used to display experimental data. By using the argument `column_dict`  you can specify which variables in the data should be displayed in which color. More info to the visualize package later. `visualize` accept a random number of keyword arguments which belong to the layout of plotlys `make_subplots` layout.

In [6]:
#constructor arguments
kwargs_exp = dict(meta_path = "metadata.xlsx"
    , types = {"off" : "offline.csv", "on": "online.CSV", "CO2" : "CO2.dat"}
    , exp_dir_manual = None
    , index_ts = {"off" : 0, "on": 0, "CO2" : 0}

    , read_csv_settings = { "off" : dict(sep=";", header = 0, usecols = None)
    , "on": dict(sep=";",encoding= "unicode_escape",decimal=",", skiprows=[1,2] , skipfooter=1, usecols = None, engine="python")
    , "CO2" : dict(sep=";",header = 0, skiprows=[0], usecols=[0,2,4], names =["ts","CO2", "p"])    }

    , to_datetime_settings = {"off" : dict(format = "%d.%m.%Y %H:%M", exact= False, errors = "coerce")
    , "on": dict(format = "%d.%m.%Y  %H:%M:%S", exact= False, errors = "coerce")
    , "CO2" : dict(format = "%d.%m.%Y %H:%M:%S", exact= False, errors = "coerce")   }

    , calc_rate = {"on" : "BASET"}
    , endpoint = "end1"
    , read_excel_settings = None
    , smooth = {"on" : "BASET_rate"}
    , kwargs_smooth = dict(halflife=3, adjust= False)
    )

# Read Fermentation 8 ("F8")
print("metadata: ")
Exp_real = Experiment(path, "F8", **kwargs_exp)
print(Exp_real.metadata)
print("experimental data: ")
display(Exp_real.dataset["on"])  #dataset contains one dataframe for one type of measurment ("on", "off", "CO2")

fig = visualize(Exp_real, column_dict= {"BASET_rate" : "cyan", "BASET_rate_smoothed" : "blue"}, title= "This is a test plot")
fig.show()


metadata: 
start            2020-12-14 09:43:00
end1             2020-12-14 18:20:00
end2             2020-12-16 10:30:00
tsfeedOn         2020-12-14 09:43:00
feed_on                          0.0
V0                               0.5
mX0_wet                          3.4
mX0                         0.914172
cX0                         1.828343
cS0                                2
drymassfactor               0.268874
mS0                              1.0
feed_rate                     0.0069
csf                              200
gas_flow                          30
T                                 32
M_base                             5
mE0                           0.0001
feed_factor                  0.00276
Name: F8, dtype: object
experimental data: 


,PDatTime,Age,TEMP,JTEMP,STIRR,pH,pO2,AIRSP,MFC_B,MFC_C,...,pHo,pO2o,EO2,ECO2,AIROV,N2SP,CO2SP,ts,BASET_rate,BASET_rate_smoothed
t,,,,,,,,,,,,,,,,,,,,,
0.037778,14.12.2020 09:45:16,0.000000,23.20500,33.8700,700.0,5.439000,102.530000,0.501150,0,0,...,-0.0002,0,0,0,0.501150,0,0,2020-12-14 09:45:16,NaN,NaN
0.121111,14.12.2020 09:50:16,0.083333,28.28625,39.6900,700.0,5.364717,93.729750,0.500850,0,0,...,-0.0002,0,0,0,0.500850,0,0,2020-12-14 09:50:16,0.000,0.000000
0.204444,14.12.2020 09:55:16,0.166667,32.55000,30.9300,700.0,5.151275,77.682583,0.500550,0,0,...,-0.0002,0,0,0,0.500550,0,0,2020-12-14 09:55:16,0.913,0.188351
0.287778,14.12.2020 10:00:16,0.250000,31.95000,32.6395,700.0,5.195275,75.889417,0.500625,0,0,...,-0.0002,0,0,0,0.500625,0,0,2020-12-14 10:00:16,1.577,0.474829
0.371111,14.12.2020 10:05:16,0.333333,31.95000,32.4975,700.0,5.194017,73.574333,0.500775,0,0,...,-0.0002,0,0,0,0.500775,0,0,2020-12-14 10:05:16,1.147,0.613497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.204444,14.12.2020 17:55:16,8.166667,31.99500,31.4025,1133.6,5.146833,40.020250,0.501150,0,0,...,-0.0002,0,0,0,0.501150,0,0,2020-12-14 17:55:16,1.551,1.510206
8.287778,14.12.2020 18:00:16,8.250000,31.99500,31.4025,1137.8,5.157725,39.618667,0.501150,0,0,...,-0.0002,0,0,0,0.501150,0,0,2020-12-14 18:00:16,1.679,1.545028
8.371111,14.12.2020 18:05:16,8.333333,31.99500,31.5000,1144.2,5.172308,39.715833,0.501150,0,0,...,-0.0002,0,0,0,0.501150,0,0,2020-12-14 18:05:16,1.581,1.552449


In [7]:
#filter manually according to time with string or pd.DateTime format, 
Exp_real.time_filter(dskey = "on", start = "14.12.2020  13:20:16", end = "14.12.2020  15:00") #In the first few hours there was a hole in the base tube, which has led to unnaturally high base consumption, so the online values are cut off before this occurence
fig = visualize(Exp_real, column_dict= {"BASET_rate" : "cyan", "BASET_rate_smoothed" : "blue"}, title= "This is a test plot", title_x = 0.5, paper_bgcolor= "lightyellow", plot_bgcolor= "orange")  #with random kayword arguments for the layout
fig.show()


### Model objects

Simulation and parameter estimation are carried out by the `Model` class which contains general functions and attributes and its subclasses e.g. `Yeast` which contain the specific model (in this case a Yeast fermentation). First we instantiate a Yeast object and print its properties, including the parameters `p`, and the yield coefficients `yields` calculated from `p` by solving linear equation systems which describe different metabolic pathways.  

The starting values of the parameters `p` are not arbitrarily chosen, they were estimated beforehand based on experimental data from a 1 L Lab fermentation of Saccharomyces cerevisiae (Bakers Yeast) from Aldi (brand: WONNEMEYER Frischbackhefe). 

In [8]:
#Instatiate yeast object with yeast class and print out its properties
y = Yeast()
print(y)
print("\n\n")
print(y.p)
print("\n\n")
print(y.yields)

Yeast()



Parameters([('qsmax', <Parameter 'qsmax', value=1.6, bounds=[0.01:5.0]>), ('qemax', <Parameter 'qemax', value=0.2361 (fixed), bounds=[0.15:0.35]>), ('base_coef', <Parameter 'base_coef', value=0.007395, bounds=[0.0001:3]>), ('qO2max', <Parameter 'qO2max', value=0.164745, bounds=[0.1:0.4]>), ('qm_max', <Parameter 'qm_max', value=0.01 (fixed), bounds=[0.0075:0.0125]>), ('Ks', <Parameter 'Ks', value=0.1 (fixed), bounds=[0.01:1]>), ('Ke', <Parameter 'Ke', value=0.1 (fixed), bounds=[0.01:1]>), ('Ki', <Parameter 'Ki', value=0.1 (fixed), bounds=[0.01:1]>), ('Yxs_ox', <Parameter 'Yxs_ox', value=0.5389, bounds=[0.3:0.6]>), ('Yxs_red', <Parameter 'Yxs_red', value=0.05 (fixed), bounds=[0.01:0.8]>), ('Yxe_et', <Parameter 'Yxe_et', value=0.72 (fixed), bounds=[0.5:0.8]>), ('Yxg_glyc', <Parameter 'Yxg_glyc', value=0.2 (fixed), bounds=[0.1:0.35]>), ('HX', <Parameter 'HX', value=1.79 (fixed), bounds=[1.77:2.1]>), ('OX', <Parameter 'OX', value=0.57 (fixed), bounds=[0.54:0.63]>), ('NX', <Parame

### Simulate
The `Yeast` class, or different `Model` classes in general, have several methods that can be used. One important method is the `simulate` method. The `simulate` method simulates data based on given information. The required information is a time grid `t_grid` on which data points should be simulated.  The models in bioprocess modeling consist often of different ordinary differential equations that are dependent among each other (ordinary differential equation System, ODE). To solve this differential equation system and to get the value for x and y from dx/dt and dy/dt we need to solve the initial value problem for the ODE. To solve an initial value problem, we need initial values x0 and y0. Those values should be defined in the initial state vector `y0` and have to be given to the simulate function. The next argument is the parameter vector `p` which contains the previously defined parameters and the control variables `c` which contain general control information like the feed_rate or the gas flow. 
To use the simulate function it is either required to give an `Experiment` object or `t_grid, p, y0, c`. When an `Experiment` object is given, the arguments `t_grid, p, y0, c` are extracted from the `Experiment` object. It is also possible to give an `Experiment` object and some of those arguments. For example, you can give an `Experiment` object and a time_grid `t_grid`, the simulate function do now use all information that it can extract from the `Experiemnt` object except from `t_grid`. The simulation is carried out now with all the information extracted from the `Experiment` object but with the manually given desired time grid `t_grid`, this can be done with all other arguments. You could also give an `Experiment`object and your own parametervector `p`.

In [9]:
from copy import deepcopy
#simulate data based on an Experiment object and a desired time grid 
t_grid = np.linspace(0,10,1000) #from 0 to 10h in 1000 steps
simulated_data = y.simulate(Exp_real, t_grid= t_grid)
visualize(simulated_data, mode_1= "lines").show()

#Now simulate with altered parameters
print("simulating with param 'qsmax' = 3: ")
p1 = deepcopy(y.p)
p1["qsmax"].value = 3
simulated_data_altered = y.simulate(Exp_real, t_grid= t_grid, p = p1)
visualize(simulated_data_altered, mode_1 = "lines")

simulating with param 'qsmax' = 3: 


It is also possible to give  a random number of keyword arguments for the Integrator `scipy.integrate.solve_ivp`. For example changing numerical parameters like the `method` (the default `method` is "Radau") or `max_step` or the relative tolerance `rtol`. 

In [10]:
simulated_data = y.simulate(Exp_real, t_grid= t_grid, method = "RK45", max_step = 1, rtol = 0.01)   #random key word arguments for scipy.integrate.solve_ivp: rtol or max_step
visualize(simulated_data, mode_1= "lines").show()

### Estimation
The biomoni package is capable of estimating one set of parameters for several experiments (e.g. `F4-F8`) with different types of measurement data (e.g. `on, off, CO2`) simultaneously. In the next example, experiment objects of five experiments are created and saved in a dictionary. Afterwards, some individual operations are carried out on some of those experiment objects (filtering data manually according to time, delete online data of some experiments because of bad settings). The yeast objects standard parameters are the ones explaining the data in `Messdaten` the best. To demonstrate a parameter estimation, we change the fit parameters to different arbitrary values with the function `change_params`. Then we start the estimation, and observe what values the function `estimate` finds.


In [11]:
#Creating dict with several experiments and process some of them
experiment_dict = {exp : Experiment(path, exp, **kwargs_exp) for exp in ["F4", "F5", "F6", "F7", "F8" ]}  #all experiments in a dictionary
experiment_dict["F8"].time_filter(dskey= "on", start = pd.to_datetime("14.12.2020  12:20:16")) #special time filter for experiment 8
[experiment_dict[exp].pop_dataframe("on") for exp in ["F4", "F5", "F6"]]   #delete online data in experiment 4,5,6 because of bad BASET_rate measurements

#Change fit parameters to arbitrary values to demonstrate estimation (will the values find back to the yeast parameters?). Value have to be within min, max
y.change_params("qsmax", value =1, max = 10)
y.change_params("base_coef", value = 1)
y.change_params("qO2max", value = 0.2)
y.change_params("Yxs_ox" , value = 0.4)
display(y.p)

name,value,initial value,min,max,vary
qsmax,1.00000000,1.6,0.01000000,10.0000000,True
qemax,0.23610000,0.2361,0.15000000,0.35000000,False
base_coef,1.00000000,0.007395,1.0000e-04,3.00000000,True
qO2max,0.20000000,0.164745,0.10000000,0.40000000,True
qm_max,0.01000000,0.01,0.00750000,0.01250000,False
Ks,0.10000000,0.1,0.01000000,1.00000000,False
Ke,0.10000000,0.1,0.01000000,1.00000000,False
Ki,0.10000000,0.1,0.01000000,1.00000000,False
Yxs_ox,0.40000000,0.5389,0.30000000,0.60000000,True
Yxs_red,0.05000000,0.05,0.01000000,0.80000000,False


 Similar to the simulate function, the parameter estimation can be performed with `Experiment` objects or with information (measurement data and settings) extracted from the `Experiment` objects. If you enter measurement data or settings manually, they will not be extracted from the `Experiment` object. It is also possible to specify the settings for solving the initial value problem in the form of a dictionary `kwargs_solve_ivp` which is given then to the `simulate` function. It is also possible to specify a random number of key word arguments `fit_kws` which are numeric parameters of the function `lmfit.minimize` which is based on `scipy.optimize`. As you can see after the estimation

In [12]:
#perform parameter estimation using all experiments simultaneously
y.estimate(experiment_dict, tau = 1)    #tau is a decay factor newer values in time have more relevance than older values, you dont have to use this factor. 
##Examples for possible key words arguments fit_kws arguments for lmfit: max_nfev (maximum function evaluations), gtol and xtol error tolerances
display(y.p)

name,value,standard error,relative error,initial value,min,max,vary
qsmax,1.57587345,0.00955102,(0.61%),1,0.01000000,10.0000000,True
qemax,0.23610000,0.00000000,(0.00%),0.2361,0.15000000,0.35000000,False
base_coef,0.00743158,3.0297e-04,(4.08%),1,1.0000e-04,3.00000000,True
qO2max,0.16264546,2.2123e-04,(0.14%),0.2,0.10000000,0.40000000,True
qm_max,0.01000000,0.00000000,(0.00%),0.01,0.00750000,0.01250000,False
Ks,0.10000000,0.00000000,(0.00%),0.1,0.01000000,1.00000000,False
Ke,0.10000000,0.00000000,(0.00%),0.1,0.01000000,1.00000000,False
Ki,0.10000000,0.00000000,(0.00%),0.1,0.01000000,1.00000000,False
Yxs_ox,0.54254002,3.7735e-06,(0.00%),0.4,0.30000000,0.60000000,True
Yxs_red,0.05000000,0.00000000,(0.00%),0.05,0.01000000,0.80000000,False


As you can see, after the estimation the standard parameters `p` saved in the `Yeast` Model are found again. The estimation works for this kind of data. If you have problems with your own individual data, the following configurations may help:
- Using different numerical setting for `scipy.integrate.solve_ivp` (`kwargs_solve_ivp`) e.g. a different method or a smaller step size in order to solve the IVP of the ODE correctly.
- Using different numerical settings for the  `lmfit.minimize` function (the estimation itself), diffetent methods (numerical algorithms) can be used.
- change your fit parameters. Maybe you have to many fit parameters on vary = True? Do some fit parameters have a strong dependence to each other, are some structurally unidentifiable? choose good start values for the fit parameters. Also the fixed parameters should have meaningful values if they contribute much to the outcome. For example `HX` should not have a value of `2000` that would not make any sense and would disrupt he outcome of the estimation. 
- In some estimations it would have been better to not use `tau` (the decay factor of the measurement data relevance).
- A parameter estimation depends on knowledge and numerical finesse.

The results of the parameter estimation can be shown with the `report` function. After the estimation, the Model object attributed called `statistics_single_exp` and `statistics_all_exp` which contain the RMSE, the BIAS and the STDDEV of either every experiment individually or of alle experiments at once

In [13]:
y.report()
print("further statistics")
display(y.stat_all)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 89
    # data points      = 2600
    # variables        = 4
    chi-square         = 5.67616843
    reduced chi-square = 0.00218651
    Akaike info crit   = -15922.1748
    Bayesian info crit = -15898.7217
[[Variables]]
    qsmax:      1.57587345 +/- 0.00955102 (0.61%) (init = 1)
    qemax:      0.2361 (fixed)
    base_coef:  0.00743158 +/- 3.0297e-04 (4.08%) (init = 1)
    qO2max:     0.16264546 +/- 2.2123e-04 (0.14%) (init = 0.2)
    qm_max:     0.01 (fixed)
    Ks:         0.1 (fixed)
    Ke:         0.1 (fixed)
    Ki:         0.1 (fixed)
    Yxs_ox:     0.54254002 +/- 3.7735e-06 (0.00%) (init = 0.4)
    Yxs_red:    0.05 (fixed)
    Yxe_et:     0.72 (fixed)
    Yxg_glyc:   0.2 (fixed)
    HX:         1.79 (fixed)
    OX:         0.57 (fixed)
    NX:         0.15 (fixed)
    g_e:        0.2163628 (fixed)
[[Correlations]] (unreported correlations are < 0.100)
    C(qsmax, Yxs_ox)  = -0.487
    C(qO2max, Yxs_

{'RMSE': {'cX': 0.44798140905347067,
  'cS': 0.39755408497404354,
  'cE': 0.388572914797322,
  'BASET': 0.24559682529796473,
  'CO2': 0.5014969393294537},
 'BIAS': {'cX': 0.051051309193666794,
  'cS': -0.02807089206670315,
  'cE': 0.05269190043905316,
  'BASET': 0.04048100659014114,
  'CO2': 0.4536510513591989},
 'STDDEV': {'cX': 0.44506303675675624,
  'cS': 0.39656181800335816,
  'cE': 0.38498373178902995,
  'BASET': 0.24223766986554326,
  'CO2': 0.21377535816249477}}

### Visualizing measurement data along with simulated data
The `visualize` module explained previously is capable of plotting two types of data (`data_1` and `data_2`). it is possible to plot the measurement data along with the simulated data generated from using the `simulate` function with estimated parameters `p` which were determined with the `estimate` function. It is possible to style the individual data with `mode_1, mode_2` and `suffix_1 , suffix_2`. You can determine which columns should be displayed in the secondary y_axis. You can hand over `Experiment` objects or dataFrames to the `visualize` function in order to visualize the data.

In [14]:
#simulate for every Experiment
t_grid = np.linspace(0,9,1001)      
sim_dict_all = {experiment.exp_id: y.simulate(experiment = experiment, t_grid = t_grid) for experiment in experiment_dict.values()}

#visualize experiments and simulated data simultaneously
for exp_id  in experiment_dict.keys():
    title = "Experiment {0}".format(exp_id) 
    visualize(experiment_dict[exp_id] , sim_dict_all[exp_id], title = title, suffix_1= "_experimental", suffix_2 = "_fitted", mode_1= "markers", mode_2= "lines", title_x= 0.5).show()

All the code above works simultaneously with other models like `Yeast_variable_feedrate` which is also a Yeast Model, just with a feedrate that changes over time.